In [1]:
!pip install -q transformers einops accelerate langchain bitsandbytes sentence_transformers sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 58.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.1/127.1 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 74.8 MB/s eta 0:00:00


In [2]:
!pip install -q pypdf
!pip install -q python-dotenv
!pip install -q joblib
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 47.0 MB/s eta 0:00:00


In [3]:
!pip install -q llama-index
!pip install -q llama-index-llms-huggingface
!pip install -q llama-index-embeddings-langchain
!pip install langchain-community langchain-core langchain-huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 78.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 66.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 73.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 42.4 MB/s eta 0:00:00


In [4]:
!pip install joblib
!pip install torch

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import json
import os
import time
import pickle
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModel
from llama_index.core import PromptTemplate, Settings
from llama_index.llms.huggingface import HuggingFaceLLM

showMessage = True

LLM_MODEL_NAME = "meta-llama/Llama-2-7b-chat-hf"

# GPU acceleration with metal on Mac
# device = torch.device("cuda")
# GPU acceleration with CUDA
device = torch.device("cpu")

# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cuda")


/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [7]:
# Function to print the current time and a message
def log_time(message):
    if showMessage:
        print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] {message}")


# Load the vector index
def load_index(directory_path):
    index_file_path = os.path.join(directory_path, "index.pkl")

    with open(index_file_path, "rb") as f:
        index = pickle.load(f)

    log_time(f"Index loaded from {index_file_path}")
    return index


# Load the model and tokenizer
def load_model(directory_path):
    model_save_path = os.path.join(directory_path, "llm_model")
    tokenizer_save_path = os.path.join(directory_path, "llm_tokenizer")

    model = AutoModelForCausalLM.from_pretrained(model_save_path)
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_save_path)

    log_time("Model and tokenizer loaded." + "██" * 10)
    return model, tokenizer


# Load the embedding model
def load_embedding_model(directory_path):
    embedding_model_path = os.path.join(directory_path, "embedding_model_cpu.pkl")

    with open(embedding_model_path, "rb") as f:
        embed_model = pickle.load(f)
        log_time("Embedding model loaded." + "██" * 10)
    return embed_model


# Load LLM configuration from JSON file
def load_config(directory_path):
    config_file_path = os.path.join(directory_path, "llm_config.json")

    with open(config_file_path, "r") as f:
        llm_config = json.load(f)
        log_time("LLM configuration loaded." + "██" * 20)
    return llm_config


# Import the saved model with HuggingFace
def initialize_llm(llm_config, model, tokenizer):
    llm = HuggingFaceLLM(
        context_window=llm_config["context_window"],
        max_new_tokens=llm_config["max_new_tokens"],
        generate_kwargs=llm_config["generate_kwargs"],
        system_prompt=llm_config["system_prompt"],
        query_wrapper_prompt=PromptTemplate("<|USER|>{query_str}<|ASSISTANT|>"),
        model=model,
        tokenizer=tokenizer,
        # tokenizer_name=LLM_MODEL_NAME,
        # model_name=LLM_MODEL_NAME,
        # device_map="auto",

    )

    log_time("LLM initialized.")
    return llm

# congfigure the settings
def configure_settings(embed_model, llm):
    Settings.embed_model = embed_model
    Settings.llm = llm
    Settings.chunk_size = 1024
    log_time("Settings configured.")

In [11]:
def generate_response(model, tokenizer, question):
    try:
        log_time("Tokenizing input...")
        inputs = tokenizer(question, return_tensors="pt").to(device)
        log_time("Input tokenized.")

        log_time("Generating response...")
        start_time = time.time()

        # with torch.no_grad():  # Disable gradient calculation for inference
              # with torch.cuda.amp.autocast():
        output = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_new_tokens=512,
            pad_token_id=tokenizer.eos_token_id,
        )
        end_time = time.time()
        log_time(f"Response generated in {end_time - start_time:.2f} seconds.")

        log_time("Decoding response...")
        response = tokenizer.decode(output[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True)
        log_time("Response decoded.")

        # Move the model back to CPU to free up GPU memory
        # model.to('cpu')
        # torch.cuda.empty_cache()

        return response
    except Exception as e:
        print(e)
        log_time(f"Failed to generate response: {e}")
        return None

In [9]:
def initialize_all(directory):
    index = load_index(directory)
    model, tokenizer = load_model(directory)
    embed_model = load_embedding_model(directory)
    llm_config = load_config(directory)
    llm = initialize_llm(llm_config, model, tokenizer)
    configure_settings(embed_model, llm)
    return model, tokenizer

# Initialize the model, tokenizer, and settings when the module is imported
model_dir = "drive/MyDrive/Model/"
model, tokenizer = initialize_all(model_dir)

[2024-06-24 03:19:11] Index loaded from drive/MyDrive/Model/index.pkl


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

[2024-06-24 03:24:08] Model and tokenizer loaded.████████████████████
[2024-06-24 03:24:17] Embedding model loaded.████████████████████


[2024-06-24 03:24:18] LLM configuration loaded.████████████████████████████████████████
[2024-06-24 03:24:18] LLM initialized.
[2024-06-24 03:24:19] Settings configured.


In [12]:
while True:
  try:
    print("Type \"exit\" to exit")
    question = input("Enter your question: ")

    if question == "exit":
      print("exiting...")
      break
    response = generate_response(model, tokenizer, question)

    print(response)
  except Exception as e:
    print(e)


Type "exit" to exit
Enter your question: What is Atrial Fibrillation?
[2024-06-24 03:26:29] Tokenizing input...
[2024-06-24 03:26:29] Input tokenized.
[2024-06-24 03:26:29] Generating response...
[2024-06-24 03:30:48] Response generated in 258.16 seconds.
[2024-06-24 03:30:48] Decoding response...
[2024-06-24 03:30:48] Response decoded.


Atrial fibrillation (AFib or AF) is a type of arrhythmia, or irregular heartbeat. It occurs when the normal electrical signals in the heart's atria (the upper chambers of the heart) become disorganized and begin to fire off randomly and quickly, out of sync with the heart's normal rhythm. This can cause the heart to beat too fast, too slow, or irregularly, leading to inefficient pumping of blood throughout the body.

AFib can be a serious condition because it can lead to blood clots, stroke, heart failure, and other complications. It is estimated that AFib affects over 33 million people worldwide and is the most common type of cardiac arrhythmia.

Cau